# CBC Padding Oracle Attack Lab

_Contributors:_

Summer 2020: Ryan Lehmkuhl, Ben Hoberman

Fall 2021: Ben Hoberman, Peyrin Kao, EvanBot

Spring 2021: Abel Yagubyan, Nicholas Ngai

Fall 2022: Jeffrey Deng, Gene Pan, Nicholas Ngai

# Introduction

Trying to implement crypto schemes yourself can be very dangerous. In the words of Runa Sandvi, The New York Times senior director of information security, "Asking why you should not roll your own crypto is a bit like asking why you should not design your own aircraft engine."

While many of the concepts we've reviewed in class may seem intuitive and simple, even the subtlest leakage of information can completely compromise any hope for confidentiality.

We will demonstrate this by completely decrypting a message encrypted with AES-CBC using a **padding oracle attack**.

# Setup

Before you get started, run the following block to install packages needed for this notebook:

In [ ]:
%pip install -r requirements.txt

In [ ]:
from IPython.display import clear_output
from helpers import pkcs7_pad, xor_block
from tests import test1, test2, test3, test4, test5

# Question 1: Padding

Recall that the AES-128 block cipher can only encrypt 16-byte messages. This means that block chaining modes such as CBC mode require the message length to be a multiple of 16. If the message we want to encrypt is not a multiple of 16, we need to add padding.

In lecture, we saw that some padding and de-padding algorithms lead to ambiguity over what your original message was. We also saw that the PKCS #7 padding algorithm (PKCS stands for Public Key Cryptography Standard) always returns your original message when de-padding.

The PKCS #7 algorithm appends the number of padding bytes to the end of the message. For example, the message `PANCAKES` is padded to become:

```
  0   1   2   3   4   5   6   7      8      9     10     11     12     13     14     15
'P' 'A' 'N' 'C' 'A' 'K' 'E' 'S' '\x08' '\x08' '\x08' '\x08' '\x08' '\x08' '\x08' '\x08'
```

**Note that if the message is a multiple of the block size, another block of all 16s (```0x10```) is added.**

**Try replacing `msg` with some different messages to get a feel for how the padding algorithm works.**

In [ ]:
### TODO: Replace the message to see how it's padded
msg = b"FROM: Kaltupia HQ // TO: All branches"

# Pads the message and splits it into blocks
padded_msg = pkcs7_pad(msg)
plaintext = [msg[i:i + 16].decode() for i in range(0, len(msg), 16)]
padded_plaintext = [padded_msg[i:i + 16] for i in range(0, len(padded_msg), 16)]

# Prints everything - decodes the blocks of byte objects to strings
print(f"Plaintext without padding: {plaintext}")
print(f"Length: {len(msg)}")
print(f"Plaintext with padding: {[x.decode() for x in padded_plaintext]}")
print(f"Length: {len(padded_msg)}\n")
print(f"Hex of padded message:")
for i in range(len(padded_plaintext)):
    print(f"Block {i}: ", " ".join('0x{:02x}'.format(c) for c in padded_plaintext[i]))

## 1.1

In Q4.5, we thought about what happens to the padding if an attacker modifies the last byte of a padded plaintext message.

**Below, choose a byte for the last byte of the padded message that will result in an invalid padding.**

In [ ]:
# Length of this message: 43
msg = b"THIS IS A TEST! Let's see if padding works!" # do not change this

### TODO: Choose a last byte (written as a number) that makes the padding invalid
invalid_last_byte = ...

# Autograder
test1(invalid_last_byte)

## 1.2

Also in Q4.5, we saw one way an attacker could change the last byte to a constant value that guarantees that the message has valid padding.

**Below, give *two* different last bytes for the padded message which will cause `valid_pad` to return `True`.**

In [ ]:
# Length of this message: 78
msg = b"As part of a new test, let's try to run a different input and see if it works!"
# do not change this

### TODO: Choose two different last bytes (written as a number) that makes the padding valid
valid_last_byte_1 = ... ### YOUR CODE HERE ###
valid_last_byte_2 = ... ### YOUR CODE HERE ###

# Autograder
test2(valid_last_byte_1, valid_last_byte_2)

# Question 2: Padding Oracles

We now know why padding is important and how to implement it, but how can a faulty implementation break our crypto system? For this, recall the **padding oracle** from Homework 2.

In cryptography, an oracle is a queryable 'black box' (a function with unknown inner-workings) which provides some piece of information which otherwise would not be available. For example, when we studied the IND-CPA game, the challenger acted as an **encryption oracle** since the adversary could query it on a given message and receive back a ciphertext without knowing how the encryption was done (i.e. which key was used). 

A padding oracle takes some ciphertext `c` as input, and returns `True` if the ciphertext (which is decrypted using the secret shared key) is properly padded and `False` otherwise. We've defined a function, `valid_pad` which acts as such a padding oracle for PKCS #7.

Many real systems naturally act as padding oracles. Consider a web server which uses AES-CBC to encrypt communications between its clients (early versions of TLS did this). If a client sends a message with invalid padding, the exception might cause the web server to respond with something like:

![An HTTP 500 status code on a web page, representing an internal server error](images/500.png)

Why is this bad? At a fundamental level, the resulting error leaks information about the plaintext which should never be allowed in any cryptographic system. But come on... detecting incorrect padding can't be that bad? Right? **WRONG.** This simple leakage ends up completely destroying any hope for confidentiality with the encryption scheme. To see why, let's review how the CBC block chaining mode works.

Recall that decryption in CBC mode is as follows:

![Block cipher diagram of CBC decryption](images/CBC-decrypt.png)

## 2.1

Assume that you have intercepted some ciphertext $(IV, C_1, C_2, \ldots, C_n)$ and have access to a padding oracle. You have complete freedom with what you send the padding oracle (ie. a subset of the ciphertext blocks or something completely different). Whatever you send, the padding oracle will decrypt it **using the original symmetric key** and truthfully report whether it is padded correctly. For now, ignore all of the blocks except for the last block $C_n$.

Notice the following property of CBC mode: When we XOR the second-to-last ciphertext block $C_{n - 1}$ with some value, that will cause the plaintext block $P_n$ to be XORed with that same value. That is to say, if we have some plaintext $P = (P_1, P_2, \dots, P_n)$ and corresponding CBC ciphertext $C = (IV, C_1, C_2, \dots, C_n)$, computing the decryption of $C' = (IV, C_1, C_2, \dots, C_{n - 1} \oplus x, C_n)$ will result in the decryption $P' = (P_1, P_2, \dots, P'_{n - 1}, P_n \oplus x)$. The second-to-last plaintext block $P'_{n - 1} = D_K(C_{n - 1} \oplus x) \oplus C_{n - 2}$ becomes gibberish, but the last plaintext block $P'_n = P_n \oplus x$ is **malleable** in a predictable fashion.

Since we have control over $C_{n - 1}$, we XOR the last byte of $C_{n - 1}$ with different values of $x$ until we find something that reports that the padding is correct. To achieve this, we want to to find some value for the last byte of $C_{n - 1}$ such that the last byte of $C_{n}$ ends up decrypting to `0x01`.

**Why would this always result in a correct padding?**

**<span style = "color: red">(YOUR ANSWER HERE)</span>**

## 2.2

Once we find the value of $x$, this gives us the following equation:

$$
\begin{aligned}
P_n \oplus x = \texttt{0x01}
\end{aligned}
$$

**Leverage the padding oracle to find the correct value of $x$ such that the last byte of $C_n$ decrypts to `0x01`. Then, use knowledge of $x$ to recover the last byte of the last block of plaintext $P_n$**.

_Hint: a brute force attack that only needs to try a small number of options is very fast. 256 is very small._

_Hint: what is special about XOR'ing a `0x00` byte and how might this interact with the oracle?_

In [ ]:
def recover_last_byte(oracle, second_last_block, last_block):
    """
    Code that finds the last byte of the last block of plaintext, given a
    padding oracle and the final two blocks of ciphertext.

    second_last_block: The second last block (C_{i-1}).
    last_block: The last block (C_{i}).
    """
    orig_last_byte = second_last_block[15]
    for x in ...: ### YOUR CODE HERE ###
        second_last_block[15] = ... ### YOUR CODE HERE ###
        if oracle(second_last_block, last_block):
            return ... ### YOUR CODE HERE ###
    return ... ### YOUR CODE HERE ###

test3(recover_last_byte)

## 2.3

Once we find a valid last byte that causes the padding oracle to return true, we want to manipulate the last 2 bytes of the second-to-last-block such that the last 2 bytes of the last block end up decrypting to `0x02`, `0x02`.

**Implement code that recovers the second-to-last-byte of the last block of plaintext $P_n$. Start by ensuring that the last byte of $C_n$ will decrypt to `0x02`, and then brute-force the second-to-last-byte.**

In [ ]:
def recover_second_last_byte(oracle, second_last_block, last_block):
    """
    Code that finds the second-to-last byte of the last block of plaintext,
    given a padding oracle and the final two blocks of ciphertext.

    second_last_block: The second last block (C_{i-1}).
    last_block: The last block (C_{i}).
    """
    orig_second_last_byte, orig_last_byte = second_last_block[14], second_last_block[15]
    second_last_block[15] = ... ### YOUR CODE HERE ###
    for x in ...: ### YOUR CODE HERE ###
        second_last_block[14] = ... ### YOUR CODE HERE ###
        if oracle(second_last_block, last_block):
             return ... ### YOUR CODE HERE ###

# Autograder
test4(recover_second_last_byte)

## 2.4

Now, we can finally implement the full decryption.

**Fill out the function below based on prior parts in order to successfully decrypt any CBC ciphertext block based on a padding oracle.**

Remember that as you discover more of the message, the "correct" padding you need to enforce will change.

In [ ]:
def decrypt_block(oracle, C_prev, C, display = False, tail = None):
    """
    Recover plaintext P_n given ciphertext C_n, C_{n-1}, and a padding oracle.
    Don't worry about the display or tail arguments -- they're for cool visualization later.

    C: current block (the one we're solving for) of the ciphertext (C_i)
    C_prev: last block of the cipher text (C_{i-1})
    oracle: a function which returns whether (C_prev, C) are padded correctly
    """
    orig_C_prev = bytearray(C_prev)
    plaintext_block = bytearray(16) # Reconstructed plaintext (P from previous parts)

    # Find the last byte of C using a previously written function.
    plaintext_block[15] = recover_last_byte(oracle, C_prev, C)
    
    # Iterate over the block (starting at the second-to-last byte) from end to beginning.
    for i in reversed(range(15)):
        # Set the padding byte that the known bytes should be to guarantee correct padding
        # What is the correct plaintext padding for a block which has n < 16 bytes of text?
        padding_byte = ... ### YOUR CODE HERE ###
        
        # Set C_prev[i + 1] to C_prev[15] to the appropriate byte for the current padding.
        for j in ...: ### YOUR CODE HERE ###
            ### YOUR CODE HERE ###
            pass
        
        # Brute-force C_prev[i].
        for x in ...: ### YOUR CODE HERE ###
            C_prev[i] = ... ### YOUR CODE HERE ###
            if oracle(C_prev, C):
                # Recover the ith byte of the plaintext block.
                plaintext_block[i] = ... ### YOUR CODE HERE ###
                
                # Visualization code -- don't worry about it
                if display:
                    progress = plaintext_block.replace(b'\x00', b' ')
                    clear_output(wait=True)
                    if tail is not None:
                        progress = progress + tail
                    print(f'Recovered so far: {progress}')
                    
                break
    return plaintext_block

# Autograder
test5(decrypt_block)

## Implementing the Attack

`Chef Brown is a famous cook and it so happens that her IoT pancake maker hosts a central web application. This allows Brown to start the machine anywhere and arrive at the canteen to a fresh stack of pancakes. Brown read somewhere that her encryption scheme should not be deterministic, so she opted to use AES-CBC with a random IV and PKCS #7 padding.`

Let's put the full attack together and execute our attack against the padding oracle! We can directly use your work from previous parts to fill out the `decrypt` function below!

In [ ]:
import recipe

# No code you have to write in here :)
def decrypt():
    # Split the ciphertext into blocks
    blocks = [recipe.iv] + [recipe.ciphertext[i:i + 16] for i in range(0, len(recipe.ciphertext), 16)]
    # This stores our recovered plaintext
    plaintext = [bytes(16) for _ in range(len(blocks))]

    # Recover each block and byte in reverse order
    for i in reversed(range(1, len(blocks))):
        C_prev = bytearray(blocks[i - 1])
        C = bytearray(blocks[i])
        plaintext[i] = decrypt_block(recipe.oracle, C_prev, C, True, b''.join(plaintext[i+1:]))
    
    return b''.join(plaintext[1:]).decode()

Run your attack:

In [ ]:
print(f"\n\nOrigin of chef Brown's recipe:\n\n{decrypt()}")

## 2.5

**What is the secret ingredient in Chef Brown's recipe?**

**<span style = "color: red">(YOUR ANSWER HERE)</span>**

# Question 3: History and Defenses

The padding oracle that you've developed in this lab has been used in plenty of real-world attacks. The attack was first discovered (publicly) in 2002. Since 2002 is well within the era of the internet, you can find the paper published by the exploit's original authors [here](https://www.iacr.org/cryptodb/archive/2002/EUROCRYPT/2850/2850.pdf). Exploits were quickly engineered against all sorts of servers to great success.

Fixes for this original attack "fixed" the vulnerability by simply stopping the server from directly telling users whether a message was padded correctly or not. But later attacks, in particular [lucky thirteen](https://arstechnica.com/information-technology/2013/02/lucky-thirteen-attack-snarfs-cookies-protected-by-ssl-encryption/), utilized the fact that a *timing* side-channel could let an attacker statistically infer when a padding was set correctly even without the server telling them explicitly. This illustrates how hard it is to avoid side-channel attacks and why we always prefer to use schemes which fail as safely as possible.

For the most part, CBC padding oracle attacks have been patched, but there are notable circumstances in which they can still be used to great success. [Some attacks](https://en.wikipedia.org/wiki/POODLE) use the fact that users can request to use older versions of protocols for backwards compatibilty, which can directly enable old attacks to come back to life. And since implementing cryptography is *hard*, patches and fixes can result in the resurrection of previously-patched vulnerabilities. Check out [this](https://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2016-2107) CVE, an ironic example of how a patch for lucky thirteen actually *enabled* a padding oracle attack.

Now, let's explore a possible defense. Sending just a message encrypted with CBC mode guarantees us confidentiality if we use a secure block cipher, which we generally assume AES is. The lack of authentication and integrity checking mean that we as attackers are allowed to freely tamper with messages in order to decrypt them. It stands to reason that added some authentication and integrity checks would prevent us from successfully modifying messages for the server's examination. For simplicity, let's consider just two possible methods of achieving this: MAC-then-encrypt and encrypt-then-MAC, both with a secure MAC algorithm.

## 3.1

Does MAC-then-encrypt prevent users from exploiting our padding oracle attack? If yes, explain why. If not, explain how to modify the attack so that it still works.

**<span style = "color: red">(YOUR ANSWER HERE)</span>**

## 3.2

Does encrypt-then-MAC prevent users from exploiting our padding oracle attack? If yes, explain why. If not, explain how to modify the attack so that it still works.

**<span style = "color: red">(YOUR ANSWER HERE)</span>**

# Submission

Submit your finished `.ipynb` notebook to the lab autograder on Gradescope. Make sure to save the notebook before you upload it!